In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import sys
import cv2
import glob
# directory reach
import pickle

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from support.ar_calculations import calculate_rotmat
from support.detect_lframe import detect_lframe_from_img
from support.pymf import get_MF_devices as get_camera_list

_folder_name = "calibration7"

In [10]:
def calculate_rotmat_from_3markers(corners, ids, camera_matrix, dist_coeffs, marker_length = 0.05):

    rotation_vector, translation_vector, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, camera_matrix, dist_coeffs)

    ids = list(ids)

    z_inx = ids.index(6)
    org_inx = ids.index(9)
    x_inx = ids.index(10)
    print(z_inx, org_inx, x_inx)

    zvec = translation_vector[z_inx][0]
    zvec = np.reshape(zvec, (3, 1))
    org = translation_vector[org_inx][0] 
    org = np.reshape(org, (3, 1))
    xvec = translation_vector[x_inx][0]
    xvec = np.reshape(xvec, (3, 1))


    translation_vector
    zvec
    rotMat = calculate_rotmat(xvec, zvec, org)

    translation_correction = np.array([0.045, -0.05, 0.045]).reshape(3, 1) # adding the corrections in the new L frame
    
    t_zvec = zvec - org 
    t_xvec = xvec - org

    rotMat.T@t_xvec + translation_correction
    
    print(rotMat.T@t_xvec, "this is t_xvec")
    print(rotMat.T@t_zvec, "this is t_zvec")
    return rotMat, org

In [11]:
"""pth to calibration files"""
_pth = os.path.join(os.path.dirname(os.getcwd()), "recording_programs","test_data", "single_cam_oct_7",_folder_name)
_pth

'c:\\Users\\Sujith\\Documents\\Projects\\skateboard_gui\\recording_programs\\test_data\\single_cam_oct_7\\calibration7'

In [12]:
"""save image in msgpack format"""

_webcam_frame_file = os.path.join(_pth, "webcam_frame.msgpack")
with open(_webcam_frame_file, "rb") as f:
    _f = mp.load(f, object_hook=mpn.decode)
_f

array([[194, 194, 194, ..., 211, 211, 211],
       [194, 194, 194, ..., 211, 211, 211],
       [194, 194, 194, ..., 211, 211, 211],
       ...,
       [124, 124, 124, ..., 154, 154, 154],
       [124, 124, 124, ..., 156, 156, 156],
       [124, 124, 124, ..., 156, 156, 156]], dtype=uint8)

In [13]:
"""for webcam calculating rotmat"""
_webcam_calib_file = os.path.join(_pth, "webcam_calibration.msgpack")
corners, ids, _ = detect_lframe_from_img(_f)
with open(_webcam_calib_file, "rb") as f:
    webcam_calib = mp.load(f, object_hook=mpn.decode)
    w_camera_matrix = webcam_calib[0]
    w_dist_coeffs = webcam_calib[1]

w_rotmat, w_org = calculate_rotmat_from_3markers(corners, ids, marker_length=0.05, camera_matrix=w_camera_matrix, dist_coeffs=w_dist_coeffs)


0 2 1
[[0.16397724]
 [0.        ]
 [0.        ]] this is t_xvec
[[0.08140614]
 [0.        ]
 [0.20939933]] this is t_zvec


In [ ]:
#save to path
_webcam_rotmat_file = os.path.join(_pth, "webcam_rotmat.msgpack")
with open(_webcam_rotmat_file, "wb") as f:
    _rt_packed = mp.packb(w_rotmat, default=mpn.encode)
    _org_packed = mp.packb(w_org, default=mpn.encode)
    f.write(_rt_packed)
    f.write(_org_packed)
    f.close()